### urllib3:

- Permite fazer requisições e extrair os dados das páginas.

In [ ]:
import urllib3

# PoolManager: Classe usada para fazer requisições http e pegar seu 
# conteúdo. Pode fazer um pool de conexões, ou seja, conexões com
# várias páginas web.
http = urllib3.PoolManager()

# Requisição para uma pagina web
pagina = http.request('GET', 'http://www.iaexpert.com.br')

# Se o código retornado for 200 a conexão funcionou
# Se o código retornado for 404 a conexão não funcionou
pagina.status

# Pegar todas informações da página (Código html)
pagina.data

### Extração de dados de HTML com Beautifullsoup

- Permite a extração mais organizada dos dados da página.

In [ ]:
from bs4 import BeautifulSoup
import urllib3

# Disabilita warnings de certificado de verificação da requisição 
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
http = urllib3.PoolManager()
pagina = http.request('GET', 'https://pt.wikipedia.org/wiki/Linguagem_de_programa%C3%A7%C3%A3o')
pagina.status

# soup = sopa (Representa todos os dados da página)
soup = BeautifulSoup(pagina.data, 'lxml')
# Retorna o(s) titulo(s) da página em formato string
soup.title.string
# Retorna todos os links da página
links = soup.find_all('a')
# Número de links encontrados
len(links)
# Mostra todos os links
for link in links:
    # Retorna o atributo o conteúdo do atributo recebido
    print(link.get('href'))
    # Retorna conteúdo (Título do link)
    print(link.contents)

### Crawler - Busca de documentos 1

- Percorre todos os links da página e adiciona todos os links em uma lista, depois abre todos os links.
- Baseado na página inicial, ele abre os links das páginas ligadas a ela.
- Faz isso para todas as páginas até que não tenha mais páginas relacionadas.

In [ ]:
import urllib3
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Importante: O urllib3 precisa da url completa, senão não funciona.

# - Busca links e trata links das páginas recebida.
# - A profundide indica o quão profundo será a busca, como ilustrado no Exemplo 1. 
# - Para uma busca muito profunda é necessário ter um computador potente e uma 
# grande banda de internet.

# Exemplo 1:
# profundidade = 1 -> Será percorrido e gerado os links apenas as lista de páginas recebidas.
# profundidade = 2 -> Será percorrido e gerado os links, tanto da lista de páginas recebidas,
# como da lista de novas páginas geradas a partir dos links gerados a partir da lista de páginas
# recebidas com parâmetro.

# Aumentar a profundidade significa repetir esse raciocíonio de maneira sucessiva.
# Se for digitado uma profundidade muito grande será gerado um looping infinito.

# Como esse é um código de teste, ele vai ser executado e não vai retorna nada.
def crawl(paginas, profundidade):
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    for i in range(profundidade):
        # É utilizado um set para evitar links repetidos
        novas_paginas = set()
        # Percorre todas as páginas passadas como parâmetro
        for pagina in paginas:
            http = urllib3.PoolManager()
            try:
                dados_pagina = http.request('GET', pagina)
            except:
                print("Erro ao abria página: " + pagina)
                continue
            
            soup = BeautifulSoup(dados_pagina.data, 'lxml')
            links = soup.find_all('a')
            # Contar quantos links válidos existem na página
            # Links válidos: links que possuem um 'href'
            contador = 0

            for link in links:
            # print(str(link.contents) + ' - ' + str(link.get('href')))

            # Mostra todos atributos de um link
            # print(link.attrs)
            # print('\n')

            # Conta links válidos
                if 'href' in link.attrs:
                    # Junta uma url base com uma url relativa
                    # Torna os links válidos, pois adiciona a parte base do
                    # inicio que estava faltando.
                    url = urljoin(pagina, str(link.get('href')))
                    # if url != link.get('href'):
                    #     print(url)
                    #     print(link.get('href'))

                    # Retira links inválidos
                    if url.find("'") != -1:
                        continue
                    
                    # print(url)
                    # Retira links que apontam para própria página
                    url = url.split('#')[0]
                    print(url)

                    # Faz mais uma verificação de controle e
                    # Armazena todos os links de cada página rodada no for
                    if url[0:4] == 'http':
                        novas_paginas.add(url)

                    contador += 1

            # Faz paginas receber novas páginas, depois que todos links
            # de todas as páginas que foram recebidas como parâmetro foram 
            # armazenados no conjunto em novas páginas.
            paginas = novas_paginas
            print(contador)
        print(len(novas_paginas))


listapaginas = ['https://pt.wikipedia.org/wiki/Linguagem_de_programa%C3%A7%C3%A3o']
crawl(listapaginas, 1)